In [76]:
import requests
from bs4 import BeautifulSoup
import random 
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
headers = {'User-Agent': ua.random}
from string import punctuation
punctuation = punctuation + '—–“”«»'
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score
stopWords = set(stopwords.words('russian'))
from collections import Counter

In [77]:
def page(url, headers):
    posts_list = []
    response = session.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    posts = soup.find_all('span', {'class': '_reachbanner_'})
    for post in posts:
        i = post.get_text()
        posts_list.append(i)
    return posts_list[:40]

In [78]:
def clean(posts):
    clean_rew = []
    for_train = []
    for_test = []
    for r in posts:
        r = r.replace('\n', '')
        r = r.replace('\r', '')
        r = r.lower()
        small_list = []
        for p in punctuation:
            if p in r:
                r = r.replace(p, ' ')
        w = nltk.word_tokenize(r)
        for elem in w:
            ana = morph.parse(elem) 
            n = ana[0].normal_form
            if n not in stopWords:
                small_list.append(n)
        clean_rew.append(small_list)
    for s in clean_rew[:30]:
        for c in s:
            for_train.append(c)
    for s in clean_rew[30:40]:
        for_test.append(s)
    return for_train, for_test

In [142]:
def unique(first_list, sec_list):
    unique_list = []
    clean_unique_list = []
    for i in first_list:
        if i not in sec_list:
            unique_list.append(i)
    for c in Counter(unique_list).items():
        if c[1]>2:
            clean_unique_list.append(c[0])
    return clean_unique_list

In [231]:
def simple_type_detect(words_list, bad_words, good_words):
    word_scores = {'negative':0, 'positive':0}
    for i in words_list:
        if i in bad_words:
            word_scores['negative'] += 1
        elif i in good_words:
            word_scores['positive'] += 1
    return Counter(word_scores).most_common(1)

In [236]:
def accuracy(tests, first_uniques, second_uniques):
    gold = []
    result = []
    for t in tests.items():
        gold.append(t[1])
        n = simple_type_detect(t[0], first_uniques, second_uniques) 
        result.append(n[0][0])
    return accuracy_score(result, gold)

In [79]:
url_bad = 'https://www.kinopoisk.ru/film/462682/reviews/ord/rating/status/bad/perpage/100/'
url_good = 'https://www.kinopoisk.ru/film/462682/reviews/ord/rating/status/good/perpage/200/'

In [107]:
bad = clean(page(url_bad, headers))

In [109]:
good = clean(page(url_good, headers))

In [88]:
test_rews = {}
for b in bad[1]:
    test_rews[tuple(b)] = 'negative'
for g in good[1]:
    test_rews[tuple(g)] = 'positive'

In [237]:
bad_uniques = unique(bad[0], good[0])

In [238]:
good_uniques = unique(good[0], bad[0])

In [239]:
accuracy(test_rews, bad_uniques, good_uniques)

0.8

In [240]:
#среди плохих тестовых рецензий правильно определяется только 6/10
#среди хороших тестовых рецензий правильно определяется 10/10
#видимо так проиходит потому что больше уникальных слов у хороших рецензий

# Как улучшить?

1. Можно воспользоваться нейросетью и обучить ее на большем количестве текстов (плохих и хороших; пусть еще смотрит не слова, а словосочетания; определяет синонимы), брать не только один фильм, и вообще брать разные типы текстов
2. Можно получше убрать шум (слова с опечатками, имена героев если они остаются и тд)